In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

import numpy as np
import matplotlib.pyplot as plt
import sys
from Bio import SeqIO
from data_generator import data_generator

#from google.colab import drive

In [19]:
large_file = "uniref50.fasta"
small_file = "100k_rows.fasta"

data_gen = data_generator(large_file, 2000)
data_gen.gen_data(1000)
print(len(data_gen.data))

1000


In [11]:
class LSTM_model():
    def __init__(self, input_size, hidden_layer_size, nr_hidden_layers, feature_size):
        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        self.nr_hidden_layers = nr_hidden_layers
        
        self.embeddings = nn.Embedding(feature_size, input_size)
        
        self.model = nn.LSTM(input_size, hidden_layer_size, nr_hidden_layers)
        
        self.linear = nn.Linear(hidden_layer_size, feature_size)
        
    def forward(self, input_data):
        embeds = self.embeddings(input_data)
        lstm_out, _ = self.model(input_data)#embeds)#.view(len(input_data), 1, -1))
        tag_space = self.linear(lstm_out)#.view(len(input_data), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [27]:
test = LSTM_model(data_gen.max_seq_len,500,1,26)

for seq in seqs:
    seq_input = prepare_seq(seq, acid_dict, max_seq_len)
    print(len(seq_input[0]))
    tag_scores = test.forward(seq_input)
    print(dir(tag_scores))
    break


#output = test.forward(char_seqs)
#output = output.detach().numpy()
#argmax = np.argmax(output[0])
#print(argmax)
#print(output[0][argmax])

ValueError: only one element tensors can be converted to Python scalars

In [ ]:

#seqs = np.random.choice(full_seqs, 100000, False)

max_length = max([len(seq) for seq in seqs])

data = torch.empty((len(seqs), max_length))
for i, seq in enumerate(seqs):
    #print(seq)
    data[i, :] = torch.from_numpy(seq)
print("done")

print("Data shape: ", data.shape)

train_samples = int(len(data) * 0.8)
train_data = torch.utils.data.TensorDataset(data[:train_samples])
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                               shuffle=True)
test_data = torch.utils.data.TensorDataset(data[train_samples:])
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                              shuffle=False)